<a href="https://colab.research.google.com/github/ahairshi/dsf-redis/blob/main/Payload_%3D_%22propertyXML%3DXML%26_securityXML%3DXML%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a common scenario: sending two different XML payloads as the **values** for two distinct form fields (`propertyXML` and `securityXML`) within a single `application/x-www-form-urlencoded` request.

Since you've previously established that your XML content comes from files and you need high throughput, I will adapt the previous script to:

1.  Read two separate XML files (e.g., `property.xml` and `security.xml`).
2.  Store their contents.
3.  Combine them into a single `data` dictionary where the keys are the field names (`propertyXML` and `securityXML`).
4.  Maintain the `FastHttpUser` and high-volume settings.

### Locust Script for Multiple XML Files as Form Values (`locustfile.py`)

This script assumes you have two XML files: **`property.xml`** and **`security.xml`**.

In [ ]:
import os
from locust import FastHttpUser, task, between, events
from typing import Dict, List, Any

# ==============================================================================
# 1. Configuration & Data Loading
# ==============================================================================

# Define the form field names and the corresponding files to load
FORM_FIELDS: Dict[str, str] = {
    "propertyXML": "property.xml",
    "securityXML": "security.xml"
}

# This dictionary will store the final loaded content {field_name: xml_content}
FORM_DATA_CONTENT: Dict[str, str] = {}

# Define the headers (Locust automatically sets Content-Type to application/x-www-form-urlencoded)
HEADERS: Dict[str, str] = {
    'Accept': '*/*',
}

# --- Event Hook to Load File Content Before Test Starts ---
@events.init.add_listener
def _(environment, **kwargs):
    """Load all XML file contents into memory once when Locust starts."""
    global FORM_DATA_CONTENT # Declare global variable to modify it

    script_dir = os.path.dirname(__file__)

    for field_name, file_name in FORM_FIELDS.items():
        try:
            file_path = os.path.join(script_dir, file_name)

            with open(file_path, 'r', encoding='utf-8') as f:
                FORM_DATA_CONTENT[field_name] = f.read()

            print(f"INFO: Loaded content for field '{field_name}' from {file_name}")

        except FileNotFoundError:
            print(f"ERROR: Payload file not found: {file_name}")
        except Exception as e:
            print(f"ERROR: Could not read file {file_name}: {e}")

# ==============================================================================
# 2. Locust User Class Definition using FastHttpUser
# ==============================================================================

class MultipleXmlFormUser(FastHttpUser):
    """
    Simulates a user posting form-urlencoded data with multiple XML values.
    """

    # !!! IMPORTANT: SET YOUR TARGET HOST HERE OR VIA COMMAND LINE !!!
    host = "http://localhost:8080"

    # Low wait time for high throughput testing (900k RPM goal)
    wait_time = between(0.01, 0.1)

    @task
    def submit_combined_xml_form(self):
        """
        Executes a POST request with the 'propertyXML' and 'securityXML' fields.
        """
        global FORM_DATA_CONTENT

        # Check if all files were loaded
        if len(FORM_DATA_CONTENT) != len(FORM_FIELDS):
            print("ERROR: Not all form content was loaded. Skipping task.")
            return

        endpoint_path = "/api/combined-xml-form"

        # FORM_DATA_CONTENT is the Python dictionary ready for the 'data' parameter
        response = self.client.post(
            endpoint_path,
            data=FORM_DATA_CONTENT,               # Passing the dict: {"propertyXML": "<xml...>", "securityXML": "<xml...>"}
            headers=HEADERS,
            name="Submit_Property_Security_XML_Form"
        )

        # --- Optional: Response Validation ---
        if response.status_code != 200:
            response.failure(f"Request failed with status code: {response.status_code}")

# ==============================================================================
# 3. HIGH-VOLUME EXECUTION NOTES
# ==============================================================================
#
# **Pre-requisite:** Create the following two files in the same directory:
# 1. 'property.xml' (content for the propertyXML field)
# 2. 'security.xml' (content for the securityXML field)
#
# Recommended Command-Line Parameters (Example for high throughput):
# locust -f locustfile.py --master --headless --users 6000 --hatch-rate 100 \
#        --run-time 1m10s --host http://your.service.com